In [2]:
import pandas as pd
import gzip
import json
import numpy as np
import matplotlib.pyplot as plt
from apyori import apriori

In [5]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [4]:


df = getDF('Office_Products_5.json.gz')

KeyboardInterrupt: 

In [7]:
df.head()

,reviewerID,asin,style,verified,reviewText,overall,summary,unixReviewTime,reviewerName,reviewTime,vote,image
0,A2NIJTYWADLK57,0140503528,{u'Format:': u' Board book'},True,kids like story BUT while i really wanted a bo...,4.0,"good story, small size book though",1510012800,cotton clay,"11 7, 2017",NaN,NaN
1,A2827D8EEURMP4,0140503528,{u'Format:': u' Hardcover'},True,Bought this used and it came in great conditio...,4.0,Good,1488844800,emankcin,"03 7, 2017",NaN,NaN
2,APB6087F4J09J,0140503528,{u'Format:': u' Board book'},True,Every story and book about Corduroy is Fantast...,5.0,Best Books for All Children,1466812800,Starbucks Fan,"06 25, 2016",NaN,NaN
3,A2DHERRZIPFU7X,0140503528,{u'Format:': u' Paperback'},True,I purchased this book for my first grade class...,5.0,Great for Math!,1456012800,Caitlyn Jacobson,"02 21, 2016",NaN,NaN
4,A2XCLJRGFANRC,0140503528,{u'Format:': u' Hardcover'},False,Having spent numerous years in an elementary s...,5.0,Love Corduroy,1438473600,E. Ervin,"08 2, 2015",NaN,NaN


In [9]:
rows,columns=df.shape
print('Number of rows: ',rows)
print('Number of columns: ',columns)

('Number of rows: ', 800357)
('Number of columns: ', 12)


In [12]:
#Dropping unused variables
df.drop(['style', 'verified', 'reviewText', 'summary', 'unixReviewTime', 'reviewTime', 'vote'
, 'image'],axis=1,inplace=True)
df.head()

,reviewerID,asin,overall,reviewerName
0,A2NIJTYWADLK57,0140503528,4.0,cotton clay
1,A2827D8EEURMP4,0140503528,4.0,emankcin
2,APB6087F4J09J,0140503528,5.0,Starbucks Fan
3,A2DHERRZIPFU7X,0140503528,5.0,Caitlyn Jacobson
4,A2XCLJRGFANRC,0140503528,5.0,E. Ervin


In [14]:
#Number of unique user id  in the data
print('Number of unique product in Raw data = ', df['asin'].nunique())
# Number of unique product id  in the data
print('Number of unique reviewer ID in Raw data = ', df['reviewerID'].nunique())
print('Number of unique reviewer name in Raw data = ', df['reviewerName'].nunique())

('Number of unique product in Raw data = ', 27965)
('Number of unique reviewer ID in Raw data = ', 101501)
('Number of unique reviewer name in Raw data = ', 81540)


In [6]:
meta_df = getDF('meta_Office_Products.json.gz')
rows,columns=meta_df.shape
print('Number of rows: ',rows)
print('Number of columns: ',columns)
meta_df.head()


('Number of rows: ', 315644)
('Number of columns: ', 18)


,category,asin,tech1,description,title,brand,rank,also_view,main_cat,similar_item,date,price,feature,also_buy,image,details,tech2,fit
0,"[Office Products, Office & School Supplies, Ed...",0012624861,"{u'Item Weight': u'10.4 ounces', u'Package Dim...",[Sequential Spelling is based on the classic O...,Sequential Spelling Level 1 Bundle with Studen...,STL Distributors,">#439,654 in Office Products (See top 100)>#3,...","[1935943065, 1935943073, B00IJH9Q4M, 002115021...",Office Products,"[{u'asin': u'', u'href': u'', u'name': u'Seque...","August 15, 2014",$32.90,NaN,NaN,NaN,NaN,NaN,NaN
1,"[Office Products, Office & School Supplies, Ed...",0078652669,NaN,"[Unusual book, , ]","Mathematics, Applications and Concepts, Course...",bailey,"3,839,628inBooks(",NaN,Books,NaN,NaN,$8.62,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0136039847,"{u'Item model number': u'American History', u'...",[Pearson MyHistoryLab Online Access Code for A...,Pearson MyHistoryLab Online Access Code for Am...,Pearson MyHistoryLab,"[>#1,925,354 in Office Products (See top 100)]",NaN,Office Products,NaN,"June 21, 2012",$0.99,[Pearson MyHistoryLab Online Access Code for A...,NaN,NaN,NaN,NaN,NaN
3,"[Office Products, Office & School Supplies, Ed...",0140503528,"{u'Item model number': u'ING0140503528', u'Num...",[Corduroy the bear goes to the launderette wit...,A Pocket for Corduroy,Ingram Book & Distributor,">#422,894 in Office Products (See top 100)>#3,...",[0140501738],Office Products,NaN,"September 14, 2006",$0.95,[9780140503524],"[0140501738, 0448421917, 0670063428, 042528875...",NaN,NaN,NaN,NaN
4,"[Office Products, Office & School Supplies, Ed...",0195396332,NaN,[WHAT EVERYONE NEEDS TO KNOW About This Series...,Social Entrepreneurship: What Everyone Needs t...,Visit Amazon's David Bornstein Page,"110,732inBooks(","[0195334760, 1586489569, 1613630328, 142210406...",Books,NaN,NaN,NaN,NaN,"[0195334760, 1613630328, 1422104060, 158648956...",NaN,NaN,NaN,NaN


In [12]:
meta_df['asin'].nunique()

306802

In [7]:
#Dropping unused variables
meta_df.drop(['tech1', 'description', 'brand', 'rank', 'main_cat', 'similar_item'
, 'date', 'feature', 'image', 'details', 'tech2', 'fit'],axis=1,inplace=True)

In [8]:
meta_df.head(10)

,category,asin,title,also_view,price,also_buy
0,"[Office Products, Office & School Supplies, Ed...",0012624861,Sequential Spelling Level 1 Bundle with Studen...,"[1935943065, 1935943073, B00IJH9Q4M, 002115021...",$32.90,NaN
1,"[Office Products, Office & School Supplies, Ed...",0078652669,"Mathematics, Applications and Concepts, Course...",NaN,$8.62,NaN
2,NaN,0136039847,Pearson MyHistoryLab Online Access Code for Am...,NaN,$0.99,NaN
3,"[Office Products, Office & School Supplies, Ed...",0140503528,A Pocket for Corduroy,[0140501738],$0.95,"[0140501738, 0448421917, 0670063428, 042528875..."
4,"[Office Products, Office & School Supplies, Ed...",0195396332,Social Entrepreneurship: What Everyone Needs t...,"[0195334760, 1586489569, 1613630328, 142210406...",NaN,"[0195334760, 1613630328, 1422104060, 158648956..."
5,"[Office Products, Office & School Supplies, Bo...",0245109919,EKLOEN Mixed Designs of Antiqued Bronze Colour...,NaN,$17.50,"[B00BLY6POE, B00G8WV5U8, 0307591662, B019XJZHQ..."
6,"[Office Products, Office & School Supplies, Pa...",0307236315,Rachael Ray's Refrigerator Notes: 3 Magnetic P...,NaN,NaN,NaN
7,"[Office Products, Office & School Supplies, Ca...",030794655X,Paulo Coelho Moments 2012 Day Planner,[1101972645],NaN,[0525435077]
8,"[Office Products, Office & School Supplies, Bo...",0310432065,Tri-Fold Organizer Black XXL Book and Bible Cover,"[0310809177, B003JAH9MU, B0007UQKO8, 031082370...",NaN,"[031043758X, 1934770132, B00W4E1TKU, 031080917..."
9,"[Office Products, Office & School Supplies, Bo...",0310520347,Adventure Bible Cover Pink Medium,"[0310727472, 0310727421, B0793FF3N5, 031080263...",$13.69,"[0310802636, 0310727510, 0310727472, 031072744..."


In [9]:
# Create also view table
# type(asin) = unicode
also_view = []
for i in range(0, rows):
    item = [meta_df.values[i, 1]]
    av = meta_df.values[i,2]
    if type(av) != float:
        item += av
    also_view.append(item)

In [10]:
# Create also buy table
also_buy = []
for i in range(0, rows):
    item = [meta_df.values[i, 1]]
    ab = meta_df.values[i, 4]
    if type(ab) != float:
        item += ab
    also_buy.append(item)

https://stackabuse.com/association-rule-mining-via-apriori-algorithm-in-python/

In [11]:
av_association_rules = apriori(also_view, min_support=0.5, min_confidence=0.4, min_lift=1, min_length=2)
av_association_results = list(av_association_rules)

KeyboardInterrupt: 

In [ ]:
ab_association_rules = apriori(also_buy, min_support=0.0005, min_confidence=0.1, min_lift=1, min_length=2)
ab_association_results = list(ab_association_rules)